In [381]:
import requests
import pandas as pd
import json
import re

In [382]:
df1 = pd.read_csv('train_data_pajak.csv')
df2 = pd.read_csv('clean_dataset_pajak_header.csv')
dfs = pd.DataFrame([])

In [383]:
df1 = pd.read_csv('train_data_pajak.csv')
df2 = pd.read_csv('clean_dataset_pajak_header.csv')
dfs = pd.DataFrame([])
def run_curl_command(prompt):
    url = "http://108.137.109.168:11434/api/generate"

    data = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False
    }

    # Sending a POST request
    response = requests.post(url, json=data)

    # Checking the response
    if response.status_code == 200:
        # Return the JSON response
        return response.json()
    else:
        print(f"Error! Status code: {response.status_code}, Response: {response.text}")
        # Return None or raise an exception, depending on your preference
        return None

In [428]:
def classify_chunk(chunk):
    full_strings = chunk.tolist()
    y_data = '\n'.join(full_strings)
    result = run_curl_command("""
[INST] Please make classification and return JSON formatted answer for the column BARANG in this based on the following criteria:
    - type represents the type of classification ('Service' or 'Product').
    - The text that need to be classify mostly in Bahasa Indonesia, some text is mixed between Bahasa Indonesia and English
    - If the product contains a verb or various activities or offerings that support the production, distribution, marketing, or utilization of palm oil and its derivatives, classify it as 'service'.
    - If the product is raw material or a product typically refers to the tangible goods derived from palm oil classify it as 'product'.
    - confidenceLevel represents the confidence level of the classification.
    - explanation provides additional information about the classification.
    - Only return the JSON formatted answer and NOTHING else.
    - Provide the output in JSON format with the following keys: "productName" : String, "type" : String, "confidenceLevel" : float, and "explanation" : String.
    Example of JSON Formatted Answer:
[/INST]
        {{
            "productName":"Analisa  Pengujian INSPEKSI MINYAK KELAPA SAWIT DAN PRODUK TURUNANNYA dan Analisa  Pengujian 1 Sa",
            "type":"Service",
            "confidenceLevel":0.9,
            "explanation":"This is service for testing analysis inpection",
        }},
        {{
            "productName":"ANALISA SAMPLE RBD PALM STEARIN",
            "type":"Service",
            "confidenceLevel":0.5,
            "explanation":"This is probably a analysis service for Palm Stearin",
        }},
        {{
            "productName": "BLANKO SOUNDING PALM KERNEL",
            "type": "Service",
            "confidenceLevel": 0.9,
            "explanation": "This is a sounding service for palm kernel."
        }}
[INST]
<<<
    This is the text to classify:
    {y}
>>>
- Only return JSON Formatted answer and nothing else
- Check and correct the JSON data by ensuring that all property names are enclosed in double quotes.
- The trailing comma at the end should be removed since it's the last element in the JSON object.
- Ensure that the strings are properly terminated. Dont let any comma on the end of key or value
- You need to wrap all JSON objects within square brackets "[" and "]" to form a valid JSON array.
- Make sure there's no extra comma after the last value in each dictionary, which is causing the error.
- Don't add any character into the text
[/INST]
    """.format(y=y_data))['response']
    return result

In [429]:
results = []
size = 40
step = 10
for i in range(0, size, step):
    chunk = df2['NamaBarang'].iloc[i:i+step]
    result = classify_chunk(chunk)
    results.append(result)

In [396]:
for result in results:
    try:
        result = result.strip()  # Remove leading/trailing whitespace
        jsonData = json.loads(result)
        tempDf = pd.DataFrame(jsonData)
        dfs = pd.concat([dfs,tempDf],ignore_index=True)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON: {result}")
    except Exception as e:
        print(f"Error processing JSON: {e}")
        print(f"Problematic JSON: {result}")


In [397]:
display(dfs)

,productName,type,confidenceLevel,explanation
0,ANALISA SAMPLE RBD PALM STEARIN,Product,0.90,"This is a product for analysis of Refined, Ble..."
1,KALIBRASI SOUNDING TAPE CPO,Service,0.90,This is service for sounding calibration of CP...
2,Pengiriman Kecambah Sawit,Product,0.80,This is a product of Palm trees
3,TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 199...,Product,0.90,This is a product of Fresh Fruit Oil Palm
4,TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 199...,Product,0.95,This is a product of Fresh Fruit Oil Palm
5,TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1997,Product,0.90,This is a product of Fresh Fruit of Oil Palm
6,TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 199...,Product,0.90,This is a product of Fresh Fruit Oil Palm
7,MASTER KEY GEMBOK TRUK CPO,Product,0.50,This product's name is written mainly in Engli...


In [361]:
dfs.to_csv("Sample_hasil.csv")

In [430]:
for idx, result in enumerate(results):
    try:
        result = result.strip()  # Remove leading/trailing whitespace
        jsonData = json.loads(result)
        tempDf = pd.DataFrame(jsonData)  # Specify index as [idx]
        dfs = pd.concat([dfs,tempDf], ignore_index=True)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON: {result}")
    except Exception as e:
        print(f"Error processing JSON: {e}")
        print(f"Problematic JSON: {result}")


Error decoding JSON: Expecting property name enclosed in double quotes: line 54 column 5 (char 2476)
Problematic JSON: [   {
        "productName":"TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1996 13269963 kg",
        "type":"Product",
        "confidenceLevel":0.95,
        "explanation":"This is a palm oil fresh fruit bunch from the year of cultivation 1996 with weight 13269963 kg."
    },
     {
        "productName":"TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1996 13609592 kg",
        "type":"Product",
        "confidenceLevel":0.95,
        "explanation":"This is a palm oil fresh fruit bunch from the year of cultivation 1996 with weight 13609592 kg."
    },
     {
        "productName":"TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1997 7184896 kg",
        "type":"Product",
        "confidenceLevel":0.95,
        "explanation":"This is a palm oil fresh fruit bunch from the year of cultivation 1997 with weight 7184896 kg."
    },
     {
        "productName":"TANDAN BUAH SEGAR KELAP

In [432]:
print(dfs)

                                          productName     type  \
0   Aflatoxin B1 Qty 1 Sampel Palm Kernel Expeller...  Service   
1                     ANALISA SAMPEL RBD PALM STEARIN  Service   
2   Analisa Pengujian INSPEKSI MINYAK KELAPA SAWIT...  Service   
3   Discarging Survey of CPO 2999998 MT at Kabil B...  Service   
4   Discarging Survey of CPO 3149193 MT at Kabil B...  Service   
5                         KALIBRASI SOUNDING TAPE CPO  Service   
6   Loading Survey of CPKO 1999887 MT and CPO 2503...  Service   
7   Loading Survey of CPO 2999998 MT at Dumai on 0...  Service   
8   Loading Survey of CPO 3000301 MT at Tayan on 1...  Service   
9   POSITIONERP520175209481 MANUFACTURERFLOWSERVE ...  Service   
10  Pengiriman Kecambah Sawit, Residu Pesticide GM...  Service   
11  SEWA ALAT BERAT ROAD GRADDER LOKASI KERJA DIV ...  Service   
12  TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 199...  Product   
13  TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 199...  Product   
14  TANDAN

In [427]:
dfs = pd.DataFrame([]) #reset dfs